In [1]:
import torch
import torch.nn as nn
from src.models import EndiannessModel
from src.dataset_loaders.ISAdetect_mips_mipsel import create_train_test_dataloaders

# Create dataloaders
mips_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mips"
mipsel_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mipsel"
train_loader, test_loader = create_train_test_dataloaders(
    mips_dir=mips_dir, mipsel_dir=mipsel_dir, test_split=0.2
)

# Train model
model = EndiannessModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 5

In [2]:
# Training loop
for epoch in range(num_epochs):
    for i, (batch_x, batch_y) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(batch_x)
        targets = batch_y.unsqueeze(1)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {i+1}, Loss: {loss.item()}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/5, Step 100, Loss: 32.29051971435547
Epoch 1/5, Loss: 31.258201599121094
Epoch 2/5, Step 100, Loss: 13.791988372802734
Epoch 2/5, Loss: 0.23960694670677185
Epoch 3/5, Step 100, Loss: 0.15108181536197662
Epoch 3/5, Loss: 0.08877663314342499
Epoch 4/5, Step 100, Loss: 0.054748114198446274
Epoch 4/5, Loss: 0.1564689725637436
Epoch 5/5, Step 100, Loss: 0.11293213069438934
Epoch 5/5, Loss: 0.023642418906092644


In [3]:
# Test model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_x, batch_y in test_loader:
        output = model(batch_x)
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0
            if pred == batch_y[i]:
                correct += 1
            total += 1
    print(f"Accuracy: {correct/total}")

Accuracy: 0.9884196185286104
